# KorniaのData AugmentationとTorchvisionの比較
GitHub  
https://github.com/kornia/kornia  
論文  
https://arxiv.org/abs/2011.09832v1  
最新Korniaドキュメント  
https://kornia.readthedocs.io/en/latest  
実装参考  
https://colab.research.google.com/github/kornia/kornia/blob/master/examples/augmentation/kornia_augmentation.ipynb  

<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/kornia_torchvision_data_augmentation_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## KorniaをGitHubからcode clone

In [ ]:
!pip install --upgrade git+https://github.com/kornia/kornia.git

In [ ]:
!pip show kornia # 作成時点 Version: 0.6.3.dev0
!pip show torchvision # 作成時点 Version: 0.11.1+cu111

## ライブラリのインストール

In [ ]:
from PIL import Image, ImageDraw, ImageFont

import torch
from torch import allclose
from torch.testing import assert_allclose

import kornia as K

from torchvision.transforms import functional as tvF
from torchvision.transforms import transforms

import matplotlib.pyplot as plt
import numpy as np

to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

def tensor_pre_transform_wrapper(input: torch.Tensor):
    """ A wrapper that tried to reproduce the actual output from:
        - transforms.ToPILImage()
        - transforms.ToTensor()
        For each image, simply (img * 255).int() // 255
    """
    return torch.round(input * 255).to(torch.uint8) / 255.

def get_compare_img(im1, im2):
    """
    Parameters
    ----------
    im1 : PILImage
        Kornia image
    im2 : PILImage
        TorchVision image
    """
    text_h = 32
    dst = Image.new('RGB', (im1.width + im2.width, im1.height+text_h))
    dst.paste(im1, (0, text_h))
    dst.paste(im2, (im1.width, text_h))

    draw = ImageDraw.Draw(dst)
    font = ImageFont.truetype('/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf', 16)

    draw.text((8,8),'Kornia',fill=(255, 255, 255), font=font)
    draw.text((im1.width+8,8),'TorchVision',fill=(255, 255, 255), font=font)

    return dst


## 画像のロード
data augumentationを試す画像をアップロードしてください。

In [ ]:
from google.colab import files
uploaded = files.upload()
uploaded = list(uploaded.keys())
uploaded_file_path = uploaded[0]
img = Image.open(uploaded_file_path)
img

## HFlip
水平方向に反転

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RHF = K.augmentation.RandomHorizontalFlip(p=1., return_transform = True)
out_tensor, _ = RHF(in_tensor)
# torchvision
out_pil = tvF.hflip(in_pil)

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## VFlip
垂直方向に反転

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RVF = K.augmentation.RandomVerticalFlip(p=1., return_transform = True)
out_tensor, _ = RVF(in_tensor)
# torchvision
out_pil = tvF.vflip(in_pil)

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## Rotate
画像を回転  
Korniaの方がエッジがやや滑らか

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

degrees = 127 #@param {type:"slider", min:0, max:360, step:1.0}
# Kornia
RR = K.augmentation.RandomRotation(p=1.,degrees=degrees, return_transform = True)
out_tensor, _ = RR(in_tensor)
# torchvision
out_pil = tvF.rotate(in_pil, angle=RR._params['degrees'].item())

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## CenterCrop
中心部分を切り抜き

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

xy = (300, 300)
# Kornia
CC = K.augmentation.CenterCrop(p=1.,size=xy, return_transform = True)
out_tensor, _ = CC(in_tensor)
# torchvision
out_pil = tvF.center_crop(in_pil, xy)

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## Erase
部分的に除去

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RE = K.augmentation.RandomErasing(
    scale=torch.tensor([0.1, 0.2]), 
    ratio=torch.tensor([0.3, 0.3]),
    p=1., return_transform = True)
out_tensor, _ = RE(in_tensor)
# torchvision
out_tv_tensor = tvF.erase(
    tensor_pre_transform_wrapper(in_tensor.unsqueeze(dim=0)).squeeze(), 
    int(RE._params["ys"].item()), int(RE._params["xs"].item()), 
    int(RE._params["heights"].item()), int(RE._params["widths"].item()),
    0
    )

concat = get_compare_img(to_pil(out_tensor.squeeze()), to_pil(out_tv_tensor.squeeze()))
concat

## Affine変換
画像の拡大縮小、回転、平行移動などを行列を使って座標を変換  
Korniaの方がエッジがやや滑らか

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RA = K.augmentation.RandomAffine(
    degrees=120.,
    translate=torch.tensor([0.1, 0.2]),
    scale=torch.tensor([0.5, 0.9]),
    shear=torch.tensor([10., 20.]),
    p=1., return_transform = True)
out_tensor, _ = RA(in_tensor)
# torchvision
out_pil = tvF.affine(
    in_pil, 
    angle=RA._params['angle'].item(), translate=RA._params['translations'].numpy()[0].tolist(),
    scale=RA._params['scale'][0][0], 
    shear=[RA._params['sx'].item(), RA._params['sy'].item()]
    )

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## 射影変換
任意の形の四角形から別の形の四角形にする変形  
Korniaの方がエッジがやや滑らか

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RP = K.augmentation.RandomPerspective(
    p=1., return_transform = True)
out_tensor, _ = RP(in_tensor)
# torchvision
out_pil = tvF.perspective(
    in_pil,
    startpoints=RP._params["start_points"].squeeze().numpy().tolist(),
    endpoints=RP._params["end_points"].squeeze().numpy().tolist()
    )

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## Grayscale
デジタル画像の中でも、ピクセルの標本値に光度以外の情報が含まれていない画像に変換

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RG = K.augmentation.RandomGrayscale(
    p=1., return_transform = True)
out_tensor, _ = RG(in_tensor)
# torchvision
out_pil = tvF.to_grayscale(in_pil)

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## コントラスト調整
Korniaは、OpenCVアプローチを採用

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

factor = 1.4 #@param {type:"slider", min:0.0, max:2.0, step:0.1}
# Kornia
# range 0~
out_tensor = K.enhance.adjust_contrast(in_tensor, float(factor))
# torchvision
# range 0~
out_pil = tvF.adjust_contrast(in_pil, float(factor))

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## 明るさ調整
Korniaは、OpenCVアプローチを採用

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

factor = 0.2 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
# Kornia
# range 0~1
out_tensor = K.enhance.adjust_brightness(in_tensor, float(factor))
# torchvision
# 0:背景、1:入力画像そのまま、1~
out_pil = tvF.adjust_brightness(in_pil, float(factor)+1.0)

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## 色相調整
赤色（R）、イエロー（Y）、緑色（G）、シアン（C）、青色（B）、マゼンタ（M）それぞれの色調を、色相環の回転方向に調整

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

factor = 0.4 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
# Kornia
# range 0~1
out_tensor = K.enhance.adjust_hue(in_tensor, float(factor))
# torchvision
# range -0.5~0.5
out_pil = tvF.adjust_hue(in_pil, float(factor)-0.5)

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## ガンマ調整
画像の階調の応答特性を示す数値の調整

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

factor = 0.2 #@param {type:"slider", min:0, max:1, step:0.1}
# Kornia
# range 0~1
out_tensor = K.enhance.adjust_gamma(in_tensor, float(factor))
# torchvision
# range 0~1
out_pil = tvF.adjust_gamma(in_pil, float(factor))

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## 彩度調整
色空間の中央軸からの距離の調整。無彩色で0

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

factor = 0.4 #@param {type:"slider", min:0.0, max:2.0, step:0.1}
# Kornia
# range 0~2
out_tensor = K.enhance.adjust_saturation(in_tensor, float(factor))
# torchvision
# range 0~2
out_pil = tvF.adjust_saturation(in_pil, float(factor))

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## ブラー
ブレ  

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RMG = K.augmentation.RandomGaussianBlur(
    kernel_size=(21, 21), 
    sigma=(2.0, 2.0),
    p=1., return_transform = True)
out_tensor, _ = RMG(in_tensor)
# torchvision
out_pil = tvF.gaussian_blur(
    in_pil, 
    kernel_size=list(RMG.flags['kernel_size']), 
    sigma=list(RMG.flags['sigma'])
    )

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## モーションブラー
動きのあるブレ  
Torchvisionは対応する機能なし

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RM = K.augmentation.RandomMotionBlur(
    kernel_size=11, angle=30.,
    direction=0.4,
    p=1., return_transform = True)
out_tensor, _ = RM(in_tensor)
# torchvision
out_pil = in_pil

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## ソラリゼーション
白黒反転

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

thresholds = 0.4 #@param {type:"slider", min:0.0, max:2.0, step:0.1}
# Kornia
RS = K.augmentation.RandomSolarize(
    thresholds=thresholds,
    p=1., return_transform = True)
out_tensor, _ = RS(in_tensor)
# torchvision
out_pil = tvF.solarize( in_pil, thresholds )

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## イコライズ
明るさ、カラー成分の平均化

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

# Kornia
RE = K.augmentation.RandomEqualize(
    p=1., return_transform = True)
out_tensor, _ = RE(in_tensor)
# torchvision
out_pil = tvF.equalize( in_pil )

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## ポスタライゼーション
トーンの連続的なグラデーションをトーンの少ないいくつかの領域に変換

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

bits=2 #@param {type:"slider", min:0, max:8, step:1}
# Kornia
out_tensor = K.enhance.posterize(in_tensor, bits)

# torchvision
out_pil = tvF.posterize( in_pil, bits )

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat

## シャープネス
画像の輪郭を調整

In [ ]:
in_tensor = to_tensor(img)
in_pil = img

sharpness=1.8 #@param {type:"slider", min:0, max:2, step:0.1}
# Kornia
out_tensor = K.enhance.sharpness(in_tensor, sharpness)

# torchvision
out_pil = tvF.adjust_sharpness( in_pil, sharpness )

concat = get_compare_img(to_pil(out_tensor.squeeze()), out_pil)
concat